In [1]:
import requests
import numpy as np
import pandas as pd
import psycopg2
import time
import datetime
import seaborn as sns
from fbprophet import Prophet
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from holidays.holiday_base import HolidayBase
import holidays
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
import itertools
from dask.distributed import Client
import plotly.offline as pyoff
import plotly.graph_objs as go


try:
    connection = psycopg2.connect(user='',
                                  password='',
                                  host="",
                                  port="",
                                  database="")
    cursor = connection.cursor()
    execute = """
    WITH CTE AS
     (
         SELECT s.sku,
                date,
                new_category,
                quantity_total,
                (quantity_il_web + quantity_il_store) AS quantity_il,
                (quantity_us_web + quantity_us_store) AS quantity_us
         FROM sales s
                  LEFT JOIN
              (
                  SELECT DISTINCT sku, new_category
                  FROM products
              ) p
              ON p.sku = s.sku
         WHERE date >= '2019-01-01'
         AND new_category IS NOT NULL
     )

SELECT date,
   new_category,
   SUM(quantity_total) AS quantity_total,
   SUM(quantity_il) AS quantity_il,
   SUM(quantity_us) AS quantity_us
FROM CTE
GROUP BY 1,2
    """
    cursor.execute(execute)
    connection.commit()
    output = cursor.fetchall()
    output = pd.DataFrame(output).rename(
        columns={0: 'date', 1: 'new_category', 2: 'quantity_total', 3: 'quantity_il', 4: 'quantity_us'})
    print("getting the latest back fill order")

except (Exception, psycopg2.Error) as error:
    print("Failed inserting record into table {}".format(error))

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

df_source = output
output = output[output['new_category'] == 'ADK']
output = output[['date','quantity_il']]
output['date'] = [d.strftime('%Y-%m-%d') for d in output['date']]
output = output.rename(columns={'date': 'ds','quantity_il': 'y'})
output.set_index('ds', inplace=True)
output['ds'] = output.index
output = output[['ds','y']]
#output['y'] += 1
output_s = output
print(output.head(20))




ModuleNotFoundError: No module named 'psycopg2'

In [2]:
year = 2010
holiday_il = []
holiday_us = []
for date, name in sorted(holidays.IL(years=range(year, year + 100, 1)).items()):
    holiday_il.append([date, name])
holiday_il = pd.DataFrame(holiday_il).rename(columns={0: 'ds', 1: 'holiday'})
print(holiday_il)

for date, name in sorted(holidays.US(years=range(year, year + 100, 1)).items()):
    holiday_us.append([date, name])
holiday_us = pd.DataFrame(holiday_us).rename(columns={0: 'ds', 1: 'holiday'})
print(holiday_us)

holidays_all = pd.concat([holiday_us, holiday_il], ignore_index=True)
print(holidays_all)

              ds           holiday
0     2010-02-26       Purim - Eve
1     2010-02-27             Purim
2     2010-02-28     Shushan Purim
3     2010-03-29  Passover I - Eve
4     2010-03-30        Passover I
...          ...               ...
3879  2109-12-21          Hanukkah
3880  2109-12-22          Hanukkah
3881  2109-12-23          Hanukkah
3882  2109-12-24          Hanukkah
3883  2109-12-25          Hanukkah

[3884 rows x 2 columns]
              ds                     holiday
0     2010-01-01              New Year's Day
1     2010-01-18  Martin Luther King Jr. Day
2     2010-02-15       Washington's Birthday
3     2010-05-31                Memorial Day
4     2010-07-04            Independence Day
...          ...                         ...
1112  2109-09-02                   Labor Day
1113  2109-10-14                Columbus Day
1114  2109-11-11                Veterans Day
1115  2109-11-28                Thanksgiving
1116  2109-12-25               Christmas Day

[1117 rows x 2

In [3]:
black_friday = []
year = 2010
for year in range(year, year + 100, 1):
    black_friday.append(pd.date_range(start=('%s'+"-11-15") % (year),end=('%s'+"-12-15") % (year)))
flat_list = []
for sublist in black_friday:
    for item in sublist:
        flat_list.append(item)

black_friday = pd.DataFrame(flat_list).rename(columns={0: 'ds'})
black_friday['ds'] = black_friday['ds'].dt.date
black_friday['holiday'] = 'Black Friday'
print(black_friday)

              ds       holiday
0     2010-11-15  Black Friday
1     2010-11-16  Black Friday
2     2010-11-17  Black Friday
3     2010-11-18  Black Friday
4     2010-11-19  Black Friday
...          ...           ...
3095  2109-12-11  Black Friday
3096  2109-12-12  Black Friday
3097  2109-12-13  Black Friday
3098  2109-12-14  Black Friday
3099  2109-12-15  Black Friday

[3100 rows x 2 columns]


In [4]:
Summer_sale = []
Winter_sale = []
year = 2010
for year in range(year, year + 100, 1):
    Winter_sale.append(pd.date_range(start=('%s'+"-01-15") % (year),end=('%s'+"-01-31") % (year)))
    Summer_sale.append(pd.date_range(start=('%s'+"-07-15") % (year),end=('%s'+"-07-31") % (year)))

Summer_sale_flat = []
Winter_sale_flat = []

for sublist1 in Winter_sale:
    for item in sublist1:
        Winter_sale_flat.append(item)
        
for sublist2 in Summer_sale:
    for item in sublist2:
        Summer_sale_flat.append(item)
        
        

Summer_sale = pd.DataFrame(Summer_sale_flat).rename(columns={0: 'ds'})
Summer_sale['ds'] = Summer_sale['ds'].dt.date
Summer_sale['holiday'] = 'Summer sale'
print(Summer_sale)

Winter_sale = pd.DataFrame(Winter_sale_flat).rename(columns={0: 'ds'})
Winter_sale['ds'] = Winter_sale['ds'].dt.date
Winter_sale['holiday'] = 'Winter sale'
print(Winter_sale)

              ds      holiday
0     2010-07-15  Summer sale
1     2010-07-16  Summer sale
2     2010-07-17  Summer sale
3     2010-07-18  Summer sale
4     2010-07-19  Summer sale
...          ...          ...
1695  2109-07-27  Summer sale
1696  2109-07-28  Summer sale
1697  2109-07-29  Summer sale
1698  2109-07-30  Summer sale
1699  2109-07-31  Summer sale

[1700 rows x 2 columns]
              ds      holiday
0     2010-01-15  Winter sale
1     2010-01-16  Winter sale
2     2010-01-17  Winter sale
3     2010-01-18  Winter sale
4     2010-01-19  Winter sale
...          ...          ...
1695  2109-01-27  Winter sale
1696  2109-01-28  Winter sale
1697  2109-01-29  Winter sale
1698  2109-01-30  Winter sale
1699  2109-01-31  Winter sale

[1700 rows x 2 columns]


In [5]:
holiday_il = pd.concat((holiday_il,black_friday,Summer_sale,Winter_sale)).sort_values(by='ds',ascending=True)
holiday_us = pd.concat((holiday_us,black_friday,Summer_sale,Winter_sale)).sort_values(by='ds',ascending=True)
print(holiday_il)

              ds      holiday
0     2010-01-15  Winter sale
1     2010-01-16  Winter sale
2     2010-01-17  Winter sale
3     2010-01-18  Winter sale
4     2010-01-19  Winter sale
...          ...          ...
3879  2109-12-21     Hanukkah
3880  2109-12-22     Hanukkah
3881  2109-12-23     Hanukkah
3882  2109-12-24     Hanukkah
3883  2109-12-25     Hanukkah

[10384 rows x 2 columns]


In [ ]:
# client = Client()  # connect to the cluster+

df = output_s
# df_cv = cross_validation(model, initial='366 days', period='90 days', horizon='425 days',
#                          parallel="processes")
param_grid = {  
    'changepoint_range': [0.01,0.1],
    'changepoint_prior_scale': [0.01,0.1],
    'seasonality_prior_scale':[1],
    'holidays_prior_scale':[1],
    'yearly_seasonality': [1,5,10],
    
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
#     control = {}
#     control['max_treedepth'] = 15
#     control['adapt_delta'] = 0.99
    m = Prophet(**params,
            interval_width=0.95,
            daily_seasonality=False,
            #seasonality_mode='multiplicative',
            holidays=holiday_il,   
#             mcmc_samples=200,
            ).fit(df)  # Fit model with given params
    df_cv = cross_validation(m, initial='366 days', period='30 days', horizon='366 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

In [ ]:
df_p = performance_metrics(df_cv)
df_p.head(90)

In [ ]:
plot_cross_validation_metric(df_cv, metric='mape')
df_p.mean()

In [ ]:
import plotly.offline as pyoff
import plotly.graph_objs as go

output = output_s
client = Client()

control = {}
control['max_treedepth'] = 15
control['adapt_delta'] = 0.99

model = Prophet(
            interval_width=0.95,
            daily_seasonality=False,
            holidays=holiday_il,
            seasonality_prior_scale=1,
            yearly_seasonality=10,
            changepoint_prior_scale=0.01,
            changepoint_range=0.1,
            holidays_prior_scale=1,
            seasonality_mode='multiplicative',
            #mcmc_samples=200
            )
model.fit(output)

forecast = model.make_future_dataframe(periods=365, freq='d',  include_history=True)
forecast = model.predict(forecast)

# Plot the forecast with the actuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=output_s['ds'], y=output_s['y'], name='Actual'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted'))

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [ ]:
mape = [] 

df_cv = cross_validation(model, initial='366 days', period='31 days', horizon='366 days', parallel="processes")
df_p = performance_metrics(df_cv, rolling_window=1)
mape.append(df_p['mape'].values[0])


In [ ]:
# Plot the components of the model
fig = model.plot_components(forecast)
plot_cross_validation_metric(df_cv, metric='mape')
df_p.mean()

In [ ]:
sum_yhat = forecast['yhat'][(forecast['ds']>='2021-01-01') & (forecast['ds']<'2021-03-01')].sum()
print('Forecast sum is:', sum_yhat)
sum_y = output_s['y'][731:790].sum()
print('Actual sum is:', sum_y)
gap = sum_yhat-sum_y
print('gap predict to actual: ',gap)

In [ ]:
# df = forecast[['ds','yhat']]
# df = df.rename(columns={'ds':'date', 'yhat':'qty_pred'})
# print(df)
# try:
#     connection = psycopg2.connect(user='',
#                                   password='',
#                                   host="",
#                                   port="",
#                                   database="")

#     records_to_insert = list(df.itertuples(index=False, name=None))
#     cursor = connection.cursor()
#     args_strings = ','.join(cursor.mogrify("(%s,%s,%s,%s)", x).decode('utf-8') for x in records_to_insert)
#     sql_insert_query = "INSERT INTO prophet VALUES " + args_strings
#     cursor.execute(sql_insert_query)
#     connection.commit()
#     print(cursor.rowcount, "Record inserted successfully into table")

# except (Exception, psycopg2.Error) as error:
#     print("Failed inserting record into table {}".format(error))

# finally:
#     # closing database connection.
#     if connection:
#         cursor.close()
#         connection.close()
#         print("PostgreSQL connection is closed")